近年来，基于Transformer结构使用海量数据自监督训练得到的预训练模型不断刷新着自然语言处理各项任务的最好成绩，同时被不断刷新的还有模型规模，大力出奇迹不再只是玩梗。不断上升的模型规模给预测部署带来了巨大困难。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/992898f7b1eb48ea95a84bfb0fe87fa74a5e16b56aa44f65a8116c88e5491a54" width=700 hspace='10'/>
</p>


模型压缩技术的发展使得这个问题得到了缓解。模型压缩能够保证一定精度的情况下，降低模型大小，进而减少推理时间，同时提升内存和计算效率。

当前模型压缩的基本方法主要包括量化、裁剪和蒸馏。量化和裁剪通常需要更为底层的支持，如低比特和稀疏矩阵运算的支持；相比之下，蒸馏是更为简单有效的模型压缩方法。通过模型蒸馏技术，ERNIE-Tiny在4倍提速的同时模型效果只有少量下降；非Transformer结构的BOW/CNN/RNN简单模型（速度成百上千倍于ERNIE模型）效果也可以有效提升。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/71b64bd7c7b24642ac0cb4428523f9fbbe8ac0ffe7b94c9a9bea4370f3454454" width=700 hspace='10'/>
</p>

该项目基于PaddleNLP围绕模型蒸馏进行实践教学，包含以下内容：
1. 如何通过PaddleNLP快速使用通用蒸馏模型ERNIE-Tiny在特定任务微调训练
2. 如何使用PaddleNLP快速实现特定任务上使用ERNIE模型蒸馏Bi-LSTM模型
3. 产出小模型后如何使用Paddle Inference进行预测部署

## 环境准备

- PaddlePaddle 安装
  
  本项目依赖于 PaddlePaddle 2.0 及以上版本，请参考[安装指南](https://www.paddlepaddle.org.cn/install/quick?docurl=/documentation/docs/zh/install/pip/windows-pip.html)进行安装

- PaddleNLP 安装

In [1]:
!pip install --upgrade paddlenlp -i https://pypi.org/simple

Requirement already up-to-date: paddlenlp in /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages (2.0.3)


## ERNIE-Tiny 微调

ERNIE-Tiny是使用ERNIE 2.0 Base模型经通用蒸馏得到的轻量级模型，蒸馏时使用蒸馏信号（知识）包括ERNIE教师模型的预测结果（logits或软标签，这能够表示更细粒度的类别概率信息，提供信息量更大的知识）和中间各层的结果（包括中间层的输出表征和attention分布）。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/2b24b5d2086448c1b20e051cdff0122d6322aee9aa774c33b2968fc48f72a58d" width=60% hspace='10'/>
</p>

它更浅（12层->3层transformer block）、更短（字粒度->subword粒度缩短输入长度），并加大宽度（768->1024 hidden size）弥补模型变浅带来的效果损失，在4倍提速的同时模型效果只有少量下降。直接使用ERNIE-Tiny在下游任务上微调即可得到兼顾效果与性能的模型。

使用PaddleNLP可以快速实现ERNIE-Tiny在特定任务上的微调训练，方法与此前实践课程中基本无二，只需将模型换成ERNIE-Tiny即可，这里只给出这些代码，更完整的内容可以参考此前教程。

In [ ]:
from paddlenlp.transformers import ErnieTinyTokenizer, ErnieForSequenceClassification

# 创建Ernie Tiny模型使用的tokenzier
tokenizer = ErnieTinyTokenizer.from_pretrained('ernie-tiny')
# 创建使用Ernie Tiny预训练模型的句子分类模型
model = ErnieForSequenceClassification.from_pretrained('ernie-tiny', num_classes=2)
# 打印查看Ernie Tiny
print(model.ernie.config['num_hidden_layers'])
print(model.ernie.config['hidden_size'])

[2021-06-24 22:56:13,825] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/vocab.txt
100%|██████████| 459/459 [00:00<00:00, 5906.20it/s]
[2021-06-24 22:56:13,968] [    INFO] - Downloading spm_cased_simp_sampled.model from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/spm_cased_simp_sampled.model
100%|██████████| 1083/1083 [00:00<00:00, 12364.80it/s]
[2021-06-24 22:56:14,113] [    INFO] - Downloading dict.wordseg.pickle from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/dict.wordseg.pickle
100%|██████████| 161822/161822 [00:03<00:00, 47640.13it/s]
[2021-06-24 22:56:23,071] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/ernie_tiny.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-tiny
[2021-06-24 22:56:23,073] [    INFO] - Downloading ernie_tiny.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_tiny/ernie_tiny.pdparams
100%|████

3
1024


## ERNIE 蒸馏 Bi-LSTM

虽然ERNIE-Tiny相比ERNIE模型有4倍提速，但这在一些性能要求苛刻的部署场景中是不够的，大量CPU和低时延的场景只能使用轻量级的BOW/CNN/RNN模型，模型蒸馏同样可以用来提升这些模型的效果。

这部分参考论文 [Distilling Task-Specific Knowledge from BERT into Simple Neural Networks](https://arxiv.org/abs/1903.12136) 实现在特定任务上使用复杂大模型蒸馏简单模型。这里的任务数据集使用中文情感分类数据集chnsenticorp；教师模型使用ERNIE微调得到的模型，学生模型使用Bi-LSTM模型；蒸馏信号（知识）使用教师模型输出的logits；另外通过数据增强扩充数据并使用教师模型预测获取更多知识。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/faef98bde6924bd6b44045f3423ea322a1e90250387b4de59b8022359a809f0e" width="50%" height=200"/>
</p>

对于教师模型的训练不再赘述，只需将第一部分微调训练中的模型换为教师模型即可，这里只对蒸馏训练学生模型的内容进行介绍。 蒸馏训练的过程也同之前按照 `数据准备 -> 模型定义 -> 优化策略 -> 迭代训练`四步说明；略有不同的只是在数据准备中加入了数据增强，在优化目标中考虑教师模型预测产生的软标签。

### 数据准备

数据准备流程如下：

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/06401adbddb84815b2d7c60ae6ad21f3eb80ee7b43bd44038ba98271f9a021b0" width="800" height=200"/>
</p>

#### 加载数据集

使用PaddleNLP提供的`load_dataset`API，即可一键完成数据集加载，返回包含了相应数据集的`MapDataset`对象。

In [ ]:
from paddlenlp.datasets import load_dataset
# 获取内置的数据集
train_ds, dev_ds = load_dataset('chnsenticorp', splits=["train", "dev"])
# 打印查看数据集部分数据
for i in range(2):
    print(train_ds[i])

100%|██████████| 1909/1909 [00:00<00:00, 45809.10it/s]


{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': ''}
{'text': '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 'label': 1, 'qid': ''}


#### 数据增强

异构网络难以直接使用网络的中间内容作为知识，考虑通过数据增强扩充数据并使用教师模型获取更多的软标签作为知识，参照论文中给出的数据增强方法，这里实现了以下两种策略：

- Masking，即以一定的概率将句子中的token替换成[MASK]。（类似图像领域的随机噪声）
- n-gram sampling，即以一定的概率从句子中采样n-gram作为样本，其中n的范围可进行设置。（类似图像领域的随机裁剪）

在实现中，数据增强基于词（whole word）粒度进行，使用jieba分词。jieba分词也是学生模型Bi-LSTM使用的tokenize方式，顺带将tokenize处理也一并在这里实现；另外教师模型ERNIE使用了不同的tokenize方式（ErnieTokenizer），结果中也将包含这两种tokenize后的数据。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/58fd83b9b4804008a3ed954d2a2599e1eb7e890916b34f8c9c41782918258edc" width="80%" height="200"/> <br />
</p>

In [ ]:
import numpy as np
import jieba
from itertools import chain
from paddlenlp.transformers import ErnieTokenizer

# 教师模型Ernie使用的tokenizer
tokenizer = ErnieTokenizer.from_pretrained("ernie-1.0")

def apply_data_augmentation(data,
                            n_iter=2,
                            p_mask=0.1,
                            p_ng=0.25,
                            ngram_range=(2, 10)):
    """
    数据增强函数
    Args:
        data：待数据增强的数据集； n_iter： 每条样本进行数据增强的次数；  p_mask：每个token被mask的概率；
        p_ng：每条样本进行n-gram采样的概率；  ngram_range：n-gram中n的区间
    """
    # 固定seed便于演示复现
    np.random.seed(2021)

    # 存放扩充后的数据
    new_data = []
    # 循环对数据集的每条样本进行数据增强
    for example in data:
        # 使用jieba分词，基于词粒度进行数据增强
        words = [word for word in jieba.cut(example['text'])]
        # 学生模型使用jieba分词结果，教师模型使用ErnieTokenizer结果
        lstm_tokens, ernie_tokens = words, tokenizer.tokenize(example['text'])
        # 将原样本数据加入扩充数据集
        new_data.append({
            "lstm_tokens": lstm_tokens,
            "ernie_tokens": ernie_tokens,
            "label": example['label']
        })
        for _ in range(n_iter):
            # 存放数据增强后学生模型和教师模型的tokenize结果
            lstm_tokens, ernie_tokens = [], []
            # 1. Masking
            for word in words:
                # 基于词粒度，词内容整体list处理
                if np.random.rand() < p_mask:
                    lstm_tokens.append(['[UNK]'])
                    ernie_tokens.append([tokenizer.mask_token])
                else:
                    lstm_tokens.append([word])
                    ernie_tokens.append(tokenizer.tokenize(word))
            # 2. N-gram sampling
            if np.random.rand() < p_ng:
                ngram_len = min(np.random.randint(ngram_range[0], ngram_range[1] + 1), len(words))
                start = np.random.randint(0, len(words) - ngram_len + 1)
                lstm_tokens = lstm_tokens[start:start + ngram_len]
                ernie_tokens = ernie_tokens[start:start + ngram_len]

            # 展开得到tokenize的结果：
            # lstm:[[房间], [太小]] -> [房间, 太小] ernie:[[房, 间], [太, 小]] -> [房, 间, 太, 小]
            lstm_tokens, ernie_tokens = list(chain(*lstm_tokens)), list(chain(*ernie_tokens))
            # 将新样本加入扩充数据集
            new_data.append({
                "lstm_tokens": lstm_tokens,
                "ernie_tokens": ernie_tokens,
                "label": example['label']
            })
    return new_data

[2021-06-24 22:58:54,254] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt
100%|██████████| 90/90 [00:00<00:00, 3385.75it/s]


实现数据增强策略之后，可以使用数据集对象的`map()`方法完成数据增强的操作。`map()`将使用当前数据集作为参数调用该函数（`batched=True`时）。

In [ ]:
# 执行数据增强，`batched=True`时执行`data_aug_fn(train_ds)`
train_ds = train_ds.map(apply_data_augmentation, batched=True)

# 打印查看数据增强后的部分结果
for i in range(2):
    print(train_ds[i])

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.804 seconds.
Prefix dict has been built successfully.


{'lstm_tokens': ['选择', '珠江', '花园', '的', '原因', '就是', '方便', '，', '有', '电动', '扶梯', '直接', '到达', '海边', '，', '周围', '餐馆', '、', '食廊', '、', '商场', '、', '超市', '、', '摊位', '一应俱全', '。', '酒店', '装修', '一般', '，', '但', '还', '算', '整洁', '。', ' ', '泳池', '在', '大堂', '的', '屋顶', '，', '因此', '很小', '，', '不过', '女儿', '倒', '是', '喜欢', '。', ' ', '包', '的', '早餐', '是', '西式', '的', '，', '还', '算', '丰富', '。', ' ', '服务', '吗', '，', '一般'], 'ernie_tokens': ['选', '择', '珠', '江', '花', '园', '的', '原', '因', '就', '是', '方', '便', '，', '有', '电', '动', '扶', '梯', '直', '接', '到', '达', '海', '边', '，', '周', '围', '餐', '馆', '、', '食', '廊', '、', '商', '场', '、', '超', '市', '、', '摊', '位', '一', '应', '俱', '全', '。', '酒', '店', '装', '修', '一', '般', '，', '但', '还', '算', '整', '洁', '。', '泳', '池', '在', '大', '堂', '的', '屋', '顶', '，', '因', '此', '很', '小', '，', '不', '过', '女', '儿', '倒', '是', '喜', '欢', '。', '包', '的', '早', '餐', '是', '西', '式', '的', '，', '还', '算', '丰', '富', '。', '服', '务', '吗', '，', '一', '般'], 'label': 1}
{'lstm_tokens': ['选择', '珠江', '花园', '的', '原因', '就是', '方便

#### tokenize 与 id 化

获得数据增强扩充的数据集后，需要将文本数据转换为模型使用的id数据，包括tokenize和id化处理。由于tokenize处理已经在数据增强过程中一并完成，下面实现中可选的进行tokenize。另外由于学生模型Bi-LSTM和教师模型ERNIE使用了不同的tokenize方式，对应也需要两个不同的转换处理。

In [ ]:
from paddlenlp.data import Vocab

# 学生模型id转换使用的词典
vocab = Vocab.load_vocabulary(
    filepath='senta_word_dict_subset.txt',
    unk_token='[UNK]',
    pad_token='[PAD]')

def convert_example_for_lstm(example, max_seq_length=128, is_tokenized=True):
    """
    将单条样本转成学生模型（Bi-LSTM）需要的输入
    """
    # tokenize分词
    words = example['lstm_tokens'] if is_tokenized else list(jieba.cut(example['text']))
    # 使用vocab转为id数据
    input_ids = [vocab[word] for word in words][:max_seq_length]
    valid_length = np.array(len(input_ids), dtype='int64')
    return input_ids, valid_length, np.array(example['label'], dtype="int64")

def convert_example_for_ernie(example, max_seq_length=128):
    """
    将单条样本转成教师模型（ERNIE）需要的输入
    """
    example = tokenizer(example['ernie_tokens'], max_seq_len=max_seq_length, is_split_into_words=True)
    return example['input_ids'], example['token_type_ids']

def convert_example_for_distill(example):
    """
    将单条样本同时转成大小模型均需要的输入
    """
    # 得到Ernie的输入
    ernie_inputs = convert_example_for_ernie(example)
    # 得到Bi-LSTM的输入
    lstm_inputs = convert_example_for_lstm(example)
    return ernie_inputs + lstm_inputs

之后同样使用数据集的`map()`方法完成转换处理。注意，不同于上面数据增强的操作，这里的转换函数作用于单条样本（对应`batched=False`）。另外这里也将转换的操作延迟到实际取数据时进行，这可以通过设置`lazy=True`实现。

In [ ]:
from functools import partial

# 训练集转换，`lazy=True`时在实际取数据时进行转换
train_ds = train_ds.map(convert_example_for_distill, lazy=True)

# 验证集转换
dev_ds = dev_ds.map(partial(convert_example_for_lstm, is_tokenized=False), lazy=True)

# 打印查看id化后的部分结果
for i in range(2):
    print(train_ds[i])

([1, 352, 790, 1252, 409, 283, 509, 5, 250, 196, 113, 10, 58, 518, 4, 9, 128, 70, 1495, 1855, 339, 293, 45, 302, 233, 554, 4, 544, 637, 1134, 774, 6, 494, 2068, 6, 278, 191, 6, 634, 99, 6, 2678, 144, 7, 149, 1573, 62, 12043, 661, 737, 371, 435, 7, 689, 4, 255, 201, 559, 407, 1308, 12043, 2275, 1110, 11, 19, 842, 5, 1207, 878, 4, 196, 198, 321, 96, 4, 16, 93, 291, 464, 1099, 10, 692, 811, 12043, 392, 5, 748, 1134, 10, 213, 220, 5, 4, 201, 559, 723, 595, 12043, 231, 112, 1114, 4, 7, 689, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [9706, 6985, 5435, 23774, 22611, 3344, 26701, 12025, 23587, 14290, 10625, 10718, 7068, 15185, 12025, 277, 12926, 8654, 13652, 8654, 1069, 8654, 18774, 8654, 13786, 17418, 2412, 7729, 9070,

#### 构造batch数据

每条样本转换为模型使用的id数据之后，还需要将数据组织成batch输入。这包括两个操作：

1. 将多条样本组成batch，这可以通过Paddle的`paddle.io.BatchSampler`接口实现。（batch_sampler组件）
2. 将batch内的数据的各字段进行补齐等操作（tensor化），这可以通过`paddlenlp.data`下的`Pad`、`Stack`等接口实现。（batchify_fn组件）

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/30e43d4659384375a2a2c1b890ca5a995c4324d7168e49cebf1d2a1e99161f7d" width=700 hspace='10'/>
</p>

在数据集、batch_sampler 组件和 batchify_fn 组件都定义完成之后，可以使用`paddle.io.DataLoader`构造batch数据的迭代器。

In [ ]:
import paddle
from paddlenlp.data import Pad, Stack, Tuple

# 定义训练集上的batch sampler
train_batch_sampler = paddle.io.BatchSampler(
    train_ds, batch_size=32, shuffle=True)
# 定义训练集上的batchify_fn，用于对每一个特征字段进行处理
train_batchify_fn = Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),       # ERNIE: input ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # ERNIE: segment ids
    Pad(axis=0, pad_val=vocab['PAD']),                 # Bi-LSTM: input ids
    Stack(dtype="int64"),                              # Bi-LSTM: sequence length
    Stack(dtype="int64")                               # label
)
# 构造训练集的dataloader
train_data_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_sampler=train_batch_sampler,
    collate_fn=train_batchify_fn)

# 定义验证集上的batch sampler
dev_batch_sampler = paddle.io.BatchSampler(
    dev_ds, batch_size=32, shuffle=False)
# 定义验证集上的batchify_fn，用于对每一个特征字段进行处理
dev_batchify_fn = Tuple(
    Pad(axis=0, pad_val=vocab['PAD']),                 # Bi-LSTM: input ids
    Stack(dtype="int64"),                              # Bi-LSTM: sequence length
    Stack(dtype="int64")                               # label
)
# 构造验证集的dataloader
dev_data_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_sampler=dev_batch_sampler,
    collate_fn=dev_batchify_fn)

到这里数据集准备就全部完成了，下一步我们需要实现模型组网。


### 模型定义

蒸馏训练中的教师模型和学生模型分别为ERNIE和Bi-LSTM

#### ERNIE 教师模型

目前PaddleNLP已经内置了包括ERNIE在内的多种基于预训练模型的常用任务的下游网络，这些网络在`paddlenlp.transformers`下，均可实现一键调用。这里直接加载提前训练好的教师模型。

In [ ]:
from paddlenlp.transformers import ErnieForSequenceClassification

# 导入已微调训练好的教师模型
teacher = ErnieForSequenceClassification.from_pretrained("/home/aistudio/data/data88390/")


#### Bi-LSTM 学生模型

参照论文中的结构进行Bi-LSTM模型实现：embedding层后经过一个双向LSTM，将两个方向最后的hidden state拼接后送入带有tanh激活函数的线性变换层，最后由分类层输出预测结果。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/043b1f35c78c4b8092f3d6ebc45564becf903458aa164f5d8e1764b44cc44889" width="50%" height="100%"/> <br />
</p>

其中a表示输入的embeddings，b表示双向LSTM，c、d分别表示反向和前向的隐藏状态，e、g表示两个全连接层，其中e中带有激活函数，f是隐藏层表示，h是网络的logit输出，i表示softmax激活函数，j是最终输出的概率。

In [ ]:
import paddle
import paddle.nn as nn
import paddle.nn.initializer as I

class BiLSTM(nn.Layer):
    def __init__(self,
                 vocab_size,
                 embed_dim,
                 hidden_size,
                 output_dim,
                 num_layers=1,
                 dropout_prob=0.0,
                 padding_idx=0,
                 init_scale=0.1):
        super(BiLSTM, self).__init__()
        self.embedder = nn.Embedding(vocab_size, embed_dim, padding_idx)

        self.lstm = nn.LSTM(
            embed_dim,
            hidden_size,
            num_layers,
            'bidirect', # 双向LSTM
            dropout=dropout_prob)

        self.fc = nn.Linear(
            hidden_size * 2,
            hidden_size,
            weight_attr=paddle.ParamAttr(initializer=I.Uniform(
                low=-init_scale, high=init_scale)))

        self.output_layer = nn.Linear(
            hidden_size,
            output_dim,
            weight_attr=paddle.ParamAttr(initializer=I.Uniform(
                low=-init_scale, high=init_scale)))

    def forward(self, x, seq_len):
        # 对文本输入接入Embedding层
        x_embed = self.embedder(x)
        # 文本表示、文本长度作为双向LSTM的输入，经过计算得到LSTM的输出和终态表示
        lstm_out, (hidden, _) = self.lstm(
            x_embed, sequence_length=seq_len)
        # 将终态两个方向最后一层的隐状态拼接在一起
        out = paddle.concat((hidden[-2, :, :], hidden[-1, :, :]), axis=1)
        # 经过一层线性层和Tanh激活函数
        out = paddle.tanh(self.fc(out))
        # 经过最后一层线性层得到logit并返回
        logits = self.output_layer(out)

        return logits

# 根据超参创建Bi-LSTM学生模型
emb_dim = 300
hidden_size = 300
vocab_size = 29496
output_dim = 2
padding_idx = 0
num_layers = 1
dropout_prob = 0.1
student = BiLSTM(vocab_size, emb_dim, hidden_size,
            output_dim, num_layers, dropout_prob,
            padding_idx)

### 优化策略与模型训练

完成模型定义后，下一步创建优化目标和优化器并迭代数据进行蒸馏训练。

相比于此前的模型训练，这里的蒸馏训练以最小化学生模型和教师模型软标签预测结果的均方误差损失为优化目标。单步训练过程也多出一个执行教师模型的前向预测获取软标签的步骤。

In [ ]:
import os

# 定义损失函数
mse_loss = nn.MSELoss()

# 定义优化器
optimizer = paddle.optimizer.Adadelta(
    learning_rate=1.0, rho=0.95, parameters=student.parameters())

# 教师模型设置为eval模式
teacher.eval()

# 定义评估指标与评估函数
metric = paddle.metric.Accuracy()
best_acc = 0
def evaluate(model, metric, data_loader):
    model.eval()
    metric.reset()
    for i, batch in enumerate(data_loader):
        input_ids, seq_len, labels = batch
        logits = model(input_ids, seq_len)
        # 每个step调用compute和update
        correct = metric.compute(logits, labels)
        metric.update(correct)
    # 最后调用accumulate得到最后的评价指标（ACC）值
    res = metric.accumulate()
    model.train()
    return res

global_step = 0
eval_freq = 200
epochs = 6
max_step = 2
for epoch in range(epochs):
    for i, batch in enumerate(train_data_loader):
        global_step += 1
        # 从data loader中取出一个batch data
        teacher_input_ids, teacher_segment_ids, student_input_ids, seq_len, labels = batch

        # 教师模型执行前向计算，获取logits作为蒸馏信号。`no_grad`下的内容将不进行梯度计算
        with paddle.no_grad():
            teacher_logits = teacher(teacher_input_ids, teacher_segment_ids)

        # 学生模型执行前向计算
        logits = student(student_input_ids, seq_len)

        # 计算学生模型和教师模型logits输出的均方误差损失
        loss = mse_loss(logits, teacher_logits)

        # 学生模型执行反向计算获取梯度，梯度计算及参数更新、优化器更新
        loss.backward()
        # 优化器更新学生模型的参数权重，然后清空梯度
        optimizer.step()
        optimizer.clear_grad()
        # 打印日志
        print("global step %d, epoch: %d, batch: %d, loss: %f" % (global_step, epoch, i, loss))
        #  对当前的模型进行评估并保存
        if global_step % eval_freq == 0:
            acc = evaluate(student, metric, dev_data_loader)
            print("accuracy at step %d: %s, " % (global_step, acc))
            if best_acc < acc:
                paddle.save(
                    student.state_dict(),
                    os.path.join("trained_models",
                        "step_" + str(global_step) + ".pdparams"))
                best_acc = acc
        if global_step == max_step: break
    if global_step == max_step: break
print("The best accuracy is: %s" % (best_acc,))

global step 1, epoch: 0, batch: 0, loss: 8.754494
global step 2, epoch: 0, batch: 1, loss: 7.164721
The best accuracy is: 0


下面是ERNIE模型、Bi-LSTM模型以及蒸馏得到的Bi-LSTM模型在ChnSentiCorp的验证集上的准确率指标：

|                   | Accuracy(%) |
| ----------------- | ----------- |
| ERNIE-1.0         | 95.55       |
| Bi-LSTM           | 92.00       |
| Distilled Bi-LSTM | 93.333      |


## 预测部署

模型训练完成之后接下来我们实现模型的预测部署。虽然训练阶段使用的动态图模式有诸多优点，包括Python风格的编程体验（使用RNN等包含控制流的网络时尤为明显）、友好的debug交互机制等。但Python动态图模式无法更好的满足预测部署阶段的性能要求，同时也限制了部署环境。

静态图是预测部署通常采用的方式。通过静态图中预先定义的网络结构，一方面无需像动态图那样执行开销较大的Python代码；另一方面，预先固定的图结构也为基于图的优化提供了可能，这些能够有效提升预测部署的性能。常用的基于图的优化策略有内存复用和算子融合，这需要预测引擎的支持。下面是算子融合的一个示例（将Transformer Block的FFN中的`矩阵乘->加bias->relu激活`替换为单个算子）：


![](https://ai-studio-static-online.cdn.bcebos.com/88ec0e7f9622453582c9f5d7a865ff8eb5aff0194b154cee875bf6af4d69a7db)


高性能预测部署需要静态图模型导出和预测引擎两方面的支持，这里分别介绍。

### 动转静导出模型

基于静态图的预测部署要求将动态图的模型转换为静态图形式的模型（网络结构和参数权重）。

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/0ddf501ed8654531abc6379d52b03b28d3eb2e928fe648e5b02d6b3a4a4e03d0" width="800" height=200"/>
</p>

Paddle静态图形式的模型（由变量和算子构成的网络结构）使用`Program`来存放，`Program`的构造可以通过Paddle的静态图模式说明，静态图模式下网络构建执行的各API会将输入输出变量和使用的算子添加到`Program`中。

In [ ]:
import paddle
# 默认为动态图模式，这里开启静态图模式
paddle.enable_static()
# 定义输入变量，静态图下变量只是一个符号化表示，并不像动态图 Tensor 那样持有实际数据
x = paddle.static.data(shape=[None, 128], dtype='float32', name='x')
linear = paddle.nn.Linear(128, 256, bias_attr=False)
# 定义计算网络，输入和输出也都是符号化表示
y = linear(x)
# 打印 program
print(paddle.static.default_main_program())
# 关闭静态图模式
paddle.disable_static()

{ // block 0
    var x : LOD_TENSOR.shape(-1, 128).dtype(float32).stop_gradient(True)
    persist trainable param linear_0.w_0 : LOD_TENSOR.shape(128, 256).dtype(float32).stop_gradient(False)
    var linear_1.tmp_0 : LOD_TENSOR.shape(-1, 256).dtype(float32).stop_gradient(False)

    {Out=['linear_1.tmp_0']} = matmul(inputs={X=['x'], Y=['linear_0.w_0']}, Scale_out = 1.0, Scale_x = 1.0, Scale_y = 1.0, alpha = 1.0, force_fp32_output = False, fused_reshape_Out = [], fused_reshape_X = [], fused_reshape_Y = [], fused_transpose_Out = [], fused_transpose_X = [], fused_transpose_Y = [], head_number = 1, mkldnn_data_type = float32, op_device = , op_namescope = /, op_role = 0, op_role_var = [], transpose_X = False, transpose_Y = False, use_mkldnn = False, use_quantizer = False)
}



结合Paddle的静态图机制，Paddle提供了从动态图模型转换并导出静态图模型（包括网络结构和参数权重）的功能，通过`jit.to_static`和`jit.save`完成。

1. `paddle.jit.to_static` 完成动态图模型到静态图模型的转换。

- 网络结构：将动态图模型的forward函数转写（重点将Python控制流转换为Paddle对应API的调用），然后以静态图模式执行，生成Program。
- 参数权重：将动态图模型的参数在生成Program时对应到其中的变量上。

动转静时还需要使用`InputSpec`提供模型输入的描述信息（shape、dtype和name）保证Program构建过程中形状和数据类型的正确性。

In [ ]:
# 设置log输出转写的代码内容
# paddle.jit.set_code_level(100)

# 加载动态图模型
param_state_dict = paddle.load("best_model_7380_933.pdparams")
student.set_state_dict(param_state_dict)

# 动转静，通过`input_spec`给出模型所需输入数据的描述，shape中的None代表可变的大小，类似上面静态图模式中的`paddle.static.data`
model = paddle.jit.to_static(
    student,
    input_spec=[
        paddle.static.InputSpec(
            shape=[None, None], dtype="int64"),  # input_ids: [batch_size, max_seq_len]
        paddle.static.InputSpec(
            shape=[None], dtype="int64")  # length: [batch_size]
    ])

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/utils.py:77: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return (isinstance(seq, collections.Sequence) and


In [ ]:
# 打印动转静产生的Program以及输入变量
print(model.forward.concrete_program.main_program)
print(model.forward.inputs)
# 打印模型参数权重内容
print(model.forward.concrete_program.parameters[0].name)
print(model.forward.concrete_program.parameters[0].value)

{ // block 0
    var x : LOD_TENSOR.shape(-1, -1).dtype(int64).stop_gradient(False)
    var seq_len : LOD_TENSOR.shape(-1,).dtype(int64).stop_gradient(False)
    persist trainable param embedding_6.w_0 : LOD_TENSOR.shape(29496, 300).dtype(float32).stop_gradient(False)
    var embedding_0.tmp_0 : LOD_TENSOR.shape(-1, -1, 300).dtype(float32).stop_gradient(False)
    persist trainable param lstm_cell_0.w_0 : LOD_TENSOR.shape(1200, 300).dtype(float32).stop_gradient(False)
    persist trainable param lstm_cell_0.w_1 : LOD_TENSOR.shape(1200, 300).dtype(float32).stop_gradient(False)
    persist trainable param lstm_cell_0.b_0 : LOD_TENSOR.shape(1200,).dtype(float32).stop_gradient(False)
    persist trainable param lstm_cell_0.b_1 : LOD_TENSOR.shape(1200,).dtype(float32).stop_gradient(False)
    persist trainable param lstm_cell_1.w_0 : LOD_TENSOR.shape(1200, 300).dtype(float32).stop_gradient(False)
    persist trainable param lstm_cell_1.w_1 : LOD_TENSOR.shape(1200, 300).dtype(float32).stop_g

2. `paddle.jit.save` 完成静态图模型（网络结构和参数权重）的序列化保存。

- 网络结构：以`.pdmodel`为扩展名的文件，可以使用visualdl来可视化。
- 参数权重：以`.pdiparams`为扩展名的文件。

In [ ]:
import os

# 保存动转静后的模型，得到 infer_model/model.pdmodel 和 infer_model/model.pdiparams 文件
paddle.jit.save(model, "infer_model/model")
os.listdir("infer_model/")

['model.pdiparams.info', 'model.pdiparams', 'model.pdmodel']


### 使用推理库预测

获得静态图模型之后，我们使用Paddle Inference进行预测部署。Paddle Inference是飞桨的原生推理库，作用于服务器端和云端，提供高性能的推理能力。

Paddle Inference采用 Predictor 进行预测。Predictor 是一个高性能预测引擎，该引擎通过对计算图的分析，完成对计算图的一系列的优化（如OP的融合、内存/显存的优化、 MKLDNN，TensorRT 等底层加速库的支持等），能够大大提升预测性能。另外Paddle Inference提供了Python、C++、GO等多语言的API，可以根据实际环境需要进行选择，为了便于演示这里使用Python API来完成，其已在安装的Paddle包中集成，直接使用即可。使用 Paddle Inference 开发 Python 预测程序仅需以下步骤：

<p align="center">
<img src="https://ai-studio-static-online.cdn.bcebos.com/f367c74dfd0b4d5b9219bfcc562fd848797bc696e6cb423189f969d7d42f79d5" width="800" height=200"/>
</p>

In [ ]:
import paddle.inference as paddle_infer

# 1. 创建配置对象，设置预测模型路径 
config = paddle_infer.Config("infer_model/model.pdmodel", "infer_model/model.pdiparams")
# 启用 GPU 进行预测 - 初始化 GPU 显存 100M, Deivce_ID 为 0
# config.enable_use_gpu(100, 0)
config.disable_gpu()
# 2. 根据配置内容创建推理引擎
predictor = paddle_infer.create_predictor(config)
# 3. 设置输入数据
# 获取输入句柄
input_handles = [
            predictor.get_input_handle(name)
            for name in predictor.get_input_names()
        ]
# 获取输入数据
data = dev_batchify_fn([dev_ds[0]])
# 设置输入数据
for input_field, input_handle in zip(data, input_handles):
    input_handle.copy_from_cpu(input_field)

# 4. 执行预测
predictor.run()

# 5. 获取预测结果
# 获取输出句柄
output_handles = [
            predictor.get_output_handle(name)
            for name in predictor.get_output_names()
        ]
# 从输出句柄获取预测结果
output = [output_handle.copy_to_cpu() for output_handle in output_handles]
# 打印预测结果
print(output)

# 打印直接使用动态图模型预测的结果
print(student(*data[:-1]).numpy())

# Predictor和动态图模型预测速度对照
import time
start_time = time.time()
for i in range(100):
    for input_field, input_handle in zip(data, input_handles):
        input_handle.copy_from_cpu(input_field)
    predictor.run()
    output = [output_handle.copy_to_cpu() for output_handle in output_handles]
print("Predictor inference time: ", time.time() - start_time)

start_time = time.time()
for i in range(100):
    output = student(*data[:-1]).numpy()
print("Dygraph model inference time: ", time.time() - start_time)


[array([[-1.6722751,  1.4945102]], dtype=float32)]
[[-1.6722751  1.4945102]]
Predictor inference time:  1.2848896980285645
Dygraph model inference time:  1.3564274311065674


### 开放性题目（参照ERNIE-1.0的压缩部署流程，改用ERNIE-Gram的Teacher模型进行蒸馏）

In [ ]:



!python -u ./run_glue.py \
    --model_type ernie-gram \
    --model_name_or_path ernie-gram-zh \
    --task_name ChnSentiCorp \
    --max_seq_length 128 \
    --batch_size 24   \
    --learning_rate 5e-5 \
    --num_train_epochs 3 \
    --logging_steps 10 \
    --save_steps 10 \
    --output_dir /home/aistudio/ernie-gram/ \
    --device gpu # or cpu



-----------  Configuration Arguments -----------
adam_epsilon: 1e-06
batch_size: 24
device: gpu
learning_rate: 5e-05
logging_steps: 10
max_seq_length: 128
max_steps: -1
model_name_or_path: ernie-gram-zh
model_type: ernie-gram
num_train_epochs: 3
output_dir: /home/aistudio/ernie-gram/
save_steps: 10
seed: 42
task_name: ChnSentiCorp
warmup_proportion: 0.1
warmup_steps: 0
weight_decay: 0.0
------------------------------------------------
2021-06-24 23:12:53,327-INFO: unique_endpoints {''}
[2021-06-24 23:12:53,356] [    INFO] - Found /home/aistudio/.paddlenlp/models/ernie-gram-zh/vocab.txt
2021-06-24 23:12:53,370-INFO: unique_endpoints {''}
[2021-06-24 23:12:53,374] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/ernie_gram_zh/ernie_gram_zh.pdparams and saved to /home/aistudio/.paddlenlp/models/ernie-gram-zh
[2021-06-24 23:12:53,374] [    INFO] - Downloading ernie_gram_zh.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/ernie_gram_zh/ernie_gram

In [2]:
from paddlenlp.transformers import ErnieGramForSequenceClassification

In [3]:
from paddlenlp.transformers import ErnieGramForSequenceClassification, ErnieGramTokenizer
# 导入新模型
teacher = ErnieGramForSequenceClassification.from_pretrained("/home/aistudio/ernie-gram/chnsenticorp_ft_model_1190.pdparams")


In [6]:
from paddlenlp.datasets import load_dataset
# 获取内置的数据集
train_ds, dev_ds = load_dataset('chnsenticorp', splits=["train", "dev"])
# 打印查看数据集部分数据
for i in range(2):
    print(train_ds[i])

{'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般', 'label': 1, 'qid': ''}
{'text': '15.4寸笔记本的键盘确实爽，基本跟台式机差不多了，蛮喜欢数字小键盘，输数字特方便，样子也很美观，做工也相当不错', 'label': 1, 'qid': ''}


In [7]:
import numpy as np
import jieba
from itertools import chain
from paddlenlp.transformers import ErnieTokenizer

# 教师模型Ernie使用的tokenizer
tokenizer = ErnieTokenizer.from_pretrained("ernie-1.0")

def apply_data_augmentation(data,
                            n_iter=2,
                            p_mask=0.1,
                            p_ng=0.25,
                            ngram_range=(2, 10)):
    """
    数据增强函数
    Args:
        data：待数据增强的数据集； n_iter： 每条样本进行数据增强的次数；  p_mask：每个token被mask的概率；
        p_ng：每条样本进行n-gram采样的概率；  ngram_range：n-gram中n的区间
    """
    # 固定seed便于演示复现
    np.random.seed(2021)

    # 存放扩充后的数据
    new_data = []
    # 循环对数据集的每条样本进行数据增强
    for example in data:
        # 使用jieba分词，基于词粒度进行数据增强
        words = [word for word in jieba.cut(example['text'])]
        # 学生模型使用jieba分词结果，教师模型使用ErnieTokenizer结果
        lstm_tokens, ernie_tokens = words, tokenizer.tokenize(example['text'])
        # 将原样本数据加入扩充数据集
        new_data.append({
            "lstm_tokens": lstm_tokens,
            "ernie_tokens": ernie_tokens,
            "label": example['label']
        })
        for _ in range(n_iter):
            # 存放数据增强后学生模型和教师模型的tokenize结果
            lstm_tokens, ernie_tokens = [], []
            # 1. Masking
            for word in words:
                # 基于词粒度，词内容整体list处理
                if np.random.rand() < p_mask:
                    lstm_tokens.append(['[UNK]'])
                    ernie_tokens.append([tokenizer.mask_token])
                else:
                    lstm_tokens.append([word])
                    ernie_tokens.append(tokenizer.tokenize(word))
            # 2. N-gram sampling
            if np.random.rand() < p_ng:
                ngram_len = min(np.random.randint(ngram_range[0], ngram_range[1] + 1), len(words))
                start = np.random.randint(0, len(words) - ngram_len + 1)
                lstm_tokens = lstm_tokens[start:start + ngram_len]
                ernie_tokens = ernie_tokens[start:start + ngram_len]

            # 展开得到tokenize的结果：
            # lstm:[[房间], [太小]] -> [房间, 太小] ernie:[[房, 间], [太, 小]] -> [房, 间, 太, 小]
            lstm_tokens, ernie_tokens = list(chain(*lstm_tokens)), list(chain(*ernie_tokens))
            # 将新样本加入扩充数据集
            new_data.append({
                "lstm_tokens": lstm_tokens,
                "ernie_tokens": ernie_tokens,
                "label": example['label']
            })
    return new_data

[2021-06-24 23:49:27,033] [    INFO] - Downloading vocab.txt from https://paddlenlp.bj.bcebos.com/models/transformers/ernie/vocab.txt
100%|██████████| 90/90 [00:00<00:00, 382.58it/s]


In [8]:
# 执行数据增强，`batched=True`时执行`data_aug_fn(train_ds)`
train_ds = train_ds.map(apply_data_augmentation, batched=True)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.808 seconds.
Prefix dict has been built successfully.


In [9]:
from paddlenlp.data import Vocab

# 学生模型id转换使用的词典
vocab = Vocab.load_vocabulary(
    filepath='senta_word_dict_subset.txt',
    unk_token='[UNK]',
    pad_token='[PAD]')

def convert_example_for_lstm(example, max_seq_length=128, is_tokenized=True):
    """
    将单条样本转成学生模型（Bi-LSTM）需要的输入
    """
    # tokenize分词
    words = example['lstm_tokens'] if is_tokenized else list(jieba.cut(example['text']))
    # 使用vocab转为id数据
    input_ids = [vocab[word] for word in words][:max_seq_length]
    valid_length = np.array(len(input_ids), dtype='int64')
    return input_ids, valid_length, np.array(example['label'], dtype="int64")

def convert_example_for_ernie(example, max_seq_length=128):
    """
    将单条样本转成教师模型（ERNIE）需要的输入
    """
    example = tokenizer(example['ernie_tokens'], max_seq_len=max_seq_length, is_split_into_words=True)
    return example['input_ids'], example['token_type_ids']

def convert_example_for_distill(example):
    """
    将单条样本同时转成大小模型均需要的输入
    """
    # 得到Ernie的输入
    ernie_inputs = convert_example_for_ernie(example)
    # 得到Bi-LSTM的输入
    lstm_inputs = convert_example_for_lstm(example)
    return ernie_inputs + lstm_inputs

In [10]:
from functools import partial

# 训练集转换，`lazy=True`时在实际取数据时进行转换
train_ds = train_ds.map(convert_example_for_distill, lazy=True)

# 验证集转换
dev_ds = dev_ds.map(partial(convert_example_for_lstm, is_tokenized=False), lazy=True)

# 打印查看id化后的部分结果
for i in range(2):
    print(train_ds[i])

([1, 352, 790, 1252, 409, 283, 509, 5, 250, 196, 113, 10, 58, 518, 4, 9, 128, 70, 1495, 1855, 339, 293, 45, 302, 233, 554, 4, 544, 637, 1134, 774, 6, 494, 2068, 6, 278, 191, 6, 634, 99, 6, 2678, 144, 7, 149, 1573, 62, 12043, 661, 737, 371, 435, 7, 689, 4, 255, 201, 559, 407, 1308, 12043, 2275, 1110, 11, 19, 842, 5, 1207, 878, 4, 196, 198, 321, 96, 4, 16, 93, 291, 464, 1099, 10, 692, 811, 12043, 392, 5, 748, 1134, 10, 213, 220, 5, 4, 201, 559, 723, 595, 12043, 231, 112, 1114, 4, 7, 689, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [9706, 6985, 5435, 23774, 22611, 3344, 26701, 12025, 23587, 14290, 10625, 10718, 7068, 15185, 12025, 277, 12926, 8654, 13652, 8654, 1069, 8654, 18774, 8654, 13786, 17418, 2412, 7729, 9070,

In [11]:
import paddle
from paddlenlp.data import Pad, Stack, Tuple

# 定义训练集上的batch sampler
train_batch_sampler = paddle.io.BatchSampler(
    train_ds, batch_size=32, shuffle=True)
# 定义训练集上的batchify_fn，用于对每一个特征字段进行处理
train_batchify_fn = Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),       # ERNIE: input ids
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),  # ERNIE: segment ids
    Pad(axis=0, pad_val=vocab['PAD']),                 # Bi-LSTM: input ids
    Stack(dtype="int64"),                              # Bi-LSTM: sequence length
    Stack(dtype="int64")                               # label
)
# 构造训练集的dataloader
train_data_loader = paddle.io.DataLoader(
    dataset=train_ds,
    batch_sampler=train_batch_sampler,
    collate_fn=train_batchify_fn)

# 定义验证集上的batch sampler
dev_batch_sampler = paddle.io.BatchSampler(
    dev_ds, batch_size=32, shuffle=False)
# 定义验证集上的batchify_fn，用于对每一个特征字段进行处理
dev_batchify_fn = Tuple(
    Pad(axis=0, pad_val=vocab['PAD']),                 # Bi-LSTM: input ids
    Stack(dtype="int64"),                              # Bi-LSTM: sequence length
    Stack(dtype="int64")                               # label
)
# 构造验证集的dataloader
dev_data_loader = paddle.io.DataLoader(
    dataset=dev_ds,
    batch_sampler=dev_batch_sampler,
    collate_fn=dev_batchify_fn)

In [22]:
import paddle
import paddle.nn as nn
import paddle.nn.initializer as I

class BiLSTM(nn.Layer):
    def __init__(self,
                 vocab_size,
                 embed_dim,
                 hidden_size,
                 output_dim,
                 num_layers=1,
                 dropout_prob=0.0,
                 padding_idx=0,
                 init_scale=0.1):
        super(BiLSTM, self).__init__()
        self.embedder = nn.Embedding(vocab_size, embed_dim, padding_idx)

        self.lstm = nn.LSTM(
            embed_dim,
            hidden_size,
            num_layers,
            'bidirect', # 双向LSTM
            dropout=dropout_prob)

        self.fc = nn.Linear(
            hidden_size * 2,
            hidden_size,
            weight_attr=paddle.ParamAttr(initializer=I.Uniform(
                low=-init_scale, high=init_scale)))

        self.output_layer = nn.Linear(
            hidden_size,
            output_dim,
            weight_attr=paddle.ParamAttr(initializer=I.Uniform(
                low=-init_scale, high=init_scale)))

    def forward(self, x, seq_len):
        # 对文本输入接入Embedding层
        x_embed = self.embedder(x)
        # 文本表示、文本长度作为双向LSTM的输入，经过计算得到LSTM的输出和终态表示
        lstm_out, (hidden, _) = self.lstm(
            x_embed, sequence_length=seq_len)
        # 将终态两个方向最后一层的隐状态拼接在一起
        out = paddle.concat((hidden[-2, :, :], hidden[-1, :, :]), axis=1)
        # 经过一层线性层和Tanh激活函数
        out = paddle.tanh(self.fc(out))
        # 经过最后一层线性层得到logit并返回
        logits = self.output_layer(out)

        return logits

# 根据超参创建Bi-LSTM学生模型
emb_dim = 300
hidden_size = 300
vocab_size = 29496
output_dim = 2
padding_idx = 0
num_layers = 1
dropout_prob = 0.1
student = BiLSTM(vocab_size, emb_dim, hidden_size,
            output_dim, num_layers, dropout_prob,
            padding_idx)

In [24]:
import os

# 定义损失函数
mse_loss = nn.MSELoss()

# 定义优化器
optimizer = paddle.optimizer.Adadelta(
    learning_rate=0.001, rho=0.95, parameters=student.parameters())

# 教师模型设置为eval模式
teacher.eval()

# 定义评估指标与评估函数
metric = paddle.metric.Accuracy()
best_acc = 0
def evaluate(model, metric, data_loader):
    model.eval()
    metric.reset()
    for i, batch in enumerate(data_loader):
        input_ids, seq_len, labels = batch
        logits = model(input_ids, seq_len)
        # 每个step调用compute和update
        correct = metric.compute(logits, labels)
        metric.update(correct)
    # 最后调用accumulate得到最后的评价指标（ACC）值
    res = metric.accumulate()
    model.train()
    return res

global_step = 0
eval_freq = 200
epochs = 6
max_step = 2
for epoch in range(epochs):
    for i, batch in enumerate(train_data_loader):
        global_step += 1
        # 从data loader中取出一个batch data
        teacher_input_ids, teacher_segment_ids, student_input_ids, seq_len, labels = batch

        # 教师模型执行前向计算，获取logits作为蒸馏信号。`no_grad`下的内容将不进行梯度计算
        with paddle.no_grad():
            teacher_logits = teacher(teacher_input_ids, teacher_segment_ids)

        # 学生模型执行前向计算
        logits = student(student_input_ids, seq_len)

        # 计算学生模型和教师模型logits输出的均方误差损失
        loss = mse_loss(logits, teacher_logits)

        # 学生模型执行反向计算获取梯度，梯度计算及参数更新、优化器更新
        loss.backward()
        # 优化器更新学生模型的参数权重，然后清空梯度
        optimizer.step()
        optimizer.clear_grad()
        # 打印日志
        print("global step %d, epoch: %d, batch: %d, loss: %f" % (global_step, epoch, i, loss))
        #  对当前的模型进行评估并保存
        if global_step % eval_freq == 0:
            acc = evaluate(student, metric, dev_data_loader)
            print("accuracy at step %d: %s, " % (global_step, acc))
            if best_acc < acc:
                paddle.save(
                    student.state_dict(),
                    os.path.join("trained_models",
                        "step_" + str(global_step) + ".pdparams"))
                best_acc = acc
        if global_step == max_step: break
    if global_step == max_step: break
print("The best accuracy is: %s" % (best_acc,))

global step 1, epoch: 0, batch: 0, loss: 4.154266
global step 2, epoch: 0, batch: 1, loss: 4.260550
The best accuracy is: 0


## github作业提交：

更为详细的内容可以从 [PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP) 获取，欢迎大家使用和贡献，也希望大家多多支持，顺手点个小小的Star~

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 